In [ ]:
# Обязательная часть
# Будем осуществлять работу с непростым набором данных о состоянии здоровья лошадей, испытывающих кишечные колики.
# Задание 1. Базовое изучение

# Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения 
# (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

# Задание 2. Работа с выбросами
# В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. 
# Принять и обосновать решение о дальнейшей работе с ними.

# Задание 3. Работа с пропусками
# Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с 
# пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('horse_data.csv', names=['surgery', 'age', 'hospital_number', 'rectal_temperature',
                                         'pulse', 'respiratory_rate', 'temperature_of_extremities', 
                                         'peripheral_pulse', 'mucous_membranes', 'capillary_refill_time', 
                                         'pain', 'peristalsis', 'abdominal_distension', 'nasogastric_tube', 
                                         'nasogastric_reflux', 'n_reflux_PH', 'rectal_examination', 
                                         'abdomen', 'packed_cell_volume', 'total_protein', 'abdom_appearance', 
                                         'abdom_total_protein', 'outcome', 'surgical_lesion', 'tol_first', 
                                         'tol_second', 'tol_third', 'cp_data'], header=None, na_values = '?')
df = df[['surgery', 'age', 'rectal_temperature', 'pulse', 
         'temperature_of_extremities', 'mucous_membranes', 'abdominal_distension', 'outcome']]
df

In [ ]:
# посмотрим на общие показатели
df.describe()

In [ ]:
df.loc[df['age'] == 9, 'age'] = 2

In [ ]:
temperature_range = df.rectal_temperature.max() - df.rectal_temperature.min()
pulse_range = df.pulse.max() - df.pulse.min()
print(f'Размах температуры: {temperature_range}')
print(f'Размах пульса: {pulse_range}')

In [ ]:
print(f'Значение моды по пульсу: {df.pulse.round().mode()[0]}, значение медианы по пульсу: {df.pulse.median()}')
print(f'Значение моды по температуре: {df.rectal_temperature.round().mode()[0]}, значение медианы по температуре: {df.rectal_temperature.median()}')

In [ ]:
# определим выбросы вручную и посмотрим, как изменились средние
print(df.pulse.median())
print(df[(df.pulse > 30) & (df.pulse < 135)].pulse.median())
print(df.pulse.mean())
print(df[(df.pulse > 30) & (df.pulse < 135)].pulse.mean())

In [ ]:
q1_pulse = df.pulse.quantile(0.25)
q3_pulse = df.pulse.quantile(0.75)
iqr_pulse = q3_pulse - q1_pulse
lower_bound = q1_pulse - (1.5 * iqr_pulse) 
upper_bound = q3_pulse + (1.5 * iqr_pulse)
remove_outliers_pulse = df[df.pulse.between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
remove_outliers_pulse

In [ ]:
q1_temp = df.rectal_temperature.quantile(0.25)
q3_temp = df.rectal_temperature.quantile(0.75)
iqr_temp = q3_temp - q1_temp
lower_bound_temp = q1_temp - (1.5 * iqr_temp) 
upper_bound_temp = q3_temp + (1.5 * iqr_temp)
remove_outliers_temp = df[df.rectal_temperature.between\
                                 (lower_bound_temp, upper_bound_temp, inclusive=True)].sort_values('rectal_temperature')
remove_outliers_temp

In [ ]:
# ищем выбросы по температуре
q1_ = df.rectal_temperature.quantile(0.25)
q3_ = df.rectal_temperature.quantile(0.75)
iqr_ = q3_ - q1_
lower_bound_ = q1_ - (1.5 * iqr_) 
upper_bound_ = q3_ + (1.5 * iqr_)

s = df[(df.rectal_temperature > upper_bound_) | (df.rectal_temperature < lower_bound_)].sort_values('rectal_temperature')
s.info()

In [ ]:
# выбросы на графике
df.rectal_temperature.plot(kind='box', title='Выбросы по ректальной температуре')

In [ ]:
# найдем выбросы по температуре
q1_p = df.pulse.quantile(0.25)
q3_p = df.pulse.quantile(0.75)
iqr_p = q3_p - q1_p
lower_bound_p = q1_p - (1.5 * iqr_p) 
upper_bound_p = q3_p + (1.5 * iqr_p)

s_p = df[(df.pulse > upper_bound_p) | (df.pulse < lower_bound_p)].sort_values('pulse')
s_p.info()

In [ ]:
# выбросы на графике
df.pulse.plot(kind='box', title='Выбросы по пульсу животного')

In [ ]:
# посмотрим долю пустых значений в отчете
for skip in df.columns:
    pct_missing = df[skip].isnull().mean()
    print(f'{skip} - {pct_missing :.1%}')

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
df.outcome.plot(kind='hist', 
                   bins=5,
                   title='Распределение результатов состояний лошадей (outcome)',
                   xticks=range(1, 4))